# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse        Shipments  \
0  Sep  2 2022  8:30AM  245569        10       CTF0011390   
1  Sep  2 2022  8:30AM  245569        10       CTF0011391   
2  Sep  2 2022  8:30AM  245573        10          8472297   
3  Sep  2 2022  8:30AM  245574        10          8472331   
4  Sep  1 2022  4:28PM  245579        12           HH6714   
5  Sep  1 2022  4:08PM  245571        20          8472308   
6  Sep  1 2022  4:07PM  245570        20          8472305   
7  Sep  1 2022  4:07PM  245568        20          8472303   
8  Sep  1 2022  4:07PM  245567        20          8472298   
9  Sep  1 2022  4:05PM  245572        16  SEQIRUS-8472307   

                        Customer ShipmentStatus  
0                 Citieffe, Inc.       Released  
1                 Citieffe, Inc.       Released  
2                 Citieffe, Inc.       Released  
3  Nextsource Biotechnology, LLC       Released  
4                      Hush Hush       Released  
5      Andelyn Biosciences, Inc.       Released  
6      Andelyn Biosciences, Inc.       Released  
7      Andelyn Biosciences, Inc.       Released  
8      Andelyn Biosciences, Inc.       Released  
9                        Seqirus       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
27  245571       Released          1
28  245572       Released          1
29  245573       Released          1
30  245574       Released          1
31  245579       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
245571                NaN        NaN       1.0
245572                NaN        NaN       1.0
245573                NaN        NaN       1.0
245574                NaN        NaN       1.0
245579                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245453                1.0        0.0       9.0
245455                0.0        3.0       3.0
245478               10.0       16.0      15.0
245525                0.0        0.0       1.0
245527                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245453                  1          0         9
245455                  0          3         3
245478                 10         16        15
245525                  0          0         1
245527                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               245453          1          0         9
1               245455          0          3         3
2               245478         10         16        15
3               245525          0          0         1
4               245527          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               245453         1                   9
1               245455                   3         3
2               245478        10        16        15
3               245525                             1
4               245527                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Sep  2 2022  8:30AM  245569        10                  Citieffe, Inc.
2   Sep  2 2022  8:30AM  245573        10                  Citieffe, Inc.
3   Sep  2 2022  8:30AM  245574        10   Nextsource Biotechnology, LLC
4   Sep  1 2022  4:28PM  245579        12                       Hush Hush
5   Sep  1 2022  4:08PM  245571        20       Andelyn Biosciences, Inc.
6   Sep  1 2022  4:07PM  245570        20       Andelyn Biosciences, Inc.
7   Sep  1 2022  4:07PM  245568        20       Andelyn Biosciences, Inc.
8   Sep  1 2022  4:07PM  245567        20       Andelyn Biosciences, Inc.
9   Sep  1 2022  4:05PM  245572        16                         Seqirus
10  Sep  1 2022  3:48PM  245565        19  Avet Pharmaceuticals Labs, Inc

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Sep  2 2022  8:30AM  245569        10                  Citieffe, Inc.   
1  Sep  2 2022  8:30AM  245573        10                  Citieffe, Inc.   
2  Sep  2 2022  8:30AM  245574        10   Nextsource Biotechnology, LLC   
3  Sep  1 2022  4:28PM  245579        12                       Hush Hush   
4  Sep  1 2022  4:08PM  245571        20       Andelyn Biosciences, Inc.   
5  Sep  1 2022  4:07PM  245570        20       Andelyn Biosciences, Inc.   
6  Sep  1 2022  4:07PM  245568        20       Andelyn Biosciences, Inc.   
7  Sep  1 2022  4:07PM  245567        20       Andelyn Biosciences, Inc.   
8  Sep  1 2022  4:05PM  245572        16                         Seqirus   
9  Sep  1 2022  3:48PM  245565        19  Avet Pharmaceuticals Labs, Inc   

  Completed Executing  Released  
0                             2  
1                             1  
2                             1  
3                             1  
4                             1  
5                             1  
6                             1  
7                             1  
8                             1  
9                             1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Sep  2 2022  8:30AM  245569        10                  Citieffe, Inc.   
1  Sep  2 2022  8:30AM  245573        10                  Citieffe, Inc.   
2  Sep  2 2022  8:30AM  245574        10   Nextsource Biotechnology, LLC   
3  Sep  1 2022  4:28PM  245579        12                       Hush Hush   
4  Sep  1 2022  4:08PM  245571        20       Andelyn Biosciences, Inc.   
5  Sep  1 2022  4:07PM  245570        20       Andelyn Biosciences, Inc.   
6  Sep  1 2022  4:07PM  245568        20       Andelyn Biosciences, Inc.   
7  Sep  1 2022  4:07PM  245567        20       Andelyn Biosciences, Inc.   
8  Sep  1 2022  4:05PM  245572        16                         Seqirus   
9  Sep  1 2022  3:48PM  245565        19  Avet Pharmaceuticals Labs, Inc   

   Released Executing Completed  
0         2                      
1         1                      
2         1                      
3         1                      
4         1                      
5         1                      
6         1                      
7         1                      
8         1                      
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Sep  2 2022  8:30AM  245569        10                 Citieffe, Inc.   
1  Sep  2 2022  8:30AM  245573        10                 Citieffe, Inc.   
2  Sep  2 2022  8:30AM  245574        10  Nextsource Biotechnology, LLC   
3  Sep  1 2022  4:28PM  245579        12                      Hush Hush   
4  Sep  1 2022  4:08PM  245571        20      Andelyn Biosciences, Inc.   

   Released Executing Completed  
0         2                      
1         1                      
2         1                      
3         1                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Sep  2 2022  8:30AM  245569        10                 Citieffe, Inc.   
1  Sep  2 2022  8:30AM  245573        10                 Citieffe, Inc.   
2  Sep  2 2022  8:30AM  245574        10  Nextsource Biotechnology, LLC   
3  Sep  1 2022  4:28PM  245579        12                      Hush Hush   
4  Sep  1 2022  4:08PM  245571        20      Andelyn Biosciences, Inc.   

   Released  Executing  Completed  
0         2        NaN        NaN  
1         1        NaN        NaN  
2         1        NaN        NaN  
3         1        NaN        NaN  
4         1        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Sep  2 2022  8:30AM  245569        10                 Citieffe, Inc.   
1  Sep  2 2022  8:30AM  245573        10                 Citieffe, Inc.   
2  Sep  2 2022  8:30AM  245574        10  Nextsource Biotechnology, LLC   
3  Sep  1 2022  4:28PM  245579        12                      Hush Hush   
4  Sep  1 2022  4:08PM  245571        20      Andelyn Biosciences, Inc.   

   Released  Executing  Completed  
0         2        0.0        0.0  
1         1        0.0        0.0  
2         1        0.0        0.0  
3         1        0.0        0.0  
4         1        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1964415        16        8.0        2.0
102         245555         1        0.0        0.0
12          245579         1        0.0        0.0
15          245478        15       16.0       10.0
16          982244         4        0.0        0.0
19          245565         1        0.0        0.0
20         2209828        39        7.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1964415        16        8.0        2.0
1       102   245555         1        0.0        0.0
2        12   245579         1        0.0        0.0
3        15   245478        15       16.0       10.0
4        16   982244         4        0.0        0.0
5        19   245565         1        0.0        0.0
6        20  2209828        39        7.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        16        8.0        2.0
1       102         1        0.0        0.0
2        12         1        0.0        0.0
3        15        15       16.0       10.0
4        16         4        0.0        0.0
5        19         1        0.0        0.0
6        20        39        7.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   16.0
1       102  Released    1.0
2        12  Released    1.0
3        15  Released   15.0
4        16  Released    4.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10  102   12    15   16   19    20
Status                                         
Completed   2.0  0.0  0.0  10.0  0.0  0.0   1.0
Executing   8.0  0.0  0.0  16.0  0.0  0.0   7.0
Released   16.0  1.0  1.0  15.0  4.0  1.0  39.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10  102   12    15   16   19    20
0          Completed   2.0  0.0  0.0  10.0  0.0  0.0   1.0
1          Executing   8.0  0.0  0.0  16.0  0.0  0.0   7.0
2           Released  16.0  1.0  1.0  15.0  4.0  1.0  39.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10  102   12    15   16   19    20
0  Completed   2.0  0.0  0.0  10.0  0.0  0.0   1.0
1  Executing   8.0  0.0  0.0  16.0  0.0  0.0   7.0
2   Released  16.0  1.0  1.0  15.0  4.0  1.0  39.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()